# Portfolio Random Path simulation from investment
#### from historical data
#### Goal is to diversify but the problem is how to distribute weight properly

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm

In [11]:
# Tickers
tickers = ['PTT.BK', 'AOT.BK', 'DELTA.BK', 'ADVANC.BK', 'KBANK.BK']

# Fetch historical data
data = yf.download(tickers, start='2023-01-01', end='2024-01-01')['Close']

# Calculate daily returns
returns = data.pct_change().dropna()

# Number of portfolios to simulate
num_portfolios = 100000

# Daily investment and total investment
daily_investment = 1000
total_days = returns.shape[0]
total_investment = daily_investment * total_days

# Array to store results and weights
results = np.zeros((6 + len(tickers), num_portfolios))  # Added len(tickers) for weights, and extra rows for total value and capital gain/loss

for i in range(num_portfolios):
    # Generate random weights
    weights = np.random.random(len(tickers))
    weights /= np.sum(weights)
    
    # Calculate expected portfolio return and scale by total investment
    portfolio_return = np.sum(weights * returns.mean()) * total_days * daily_investment
    
    # Calculate expected portfolio volatility and scale by total investment
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * total_days, weights))) * daily_investment
    
    # Calculate portfolio VaR: Value at Risk at 95% confidence level, and scale by total investment
    portfolio_var = norm.ppf(0.05, portfolio_return, portfolio_stddev)
    
    # Store results in the results array
    results[0,i] = portfolio_return
    results[1,i] = portfolio_stddev
    results[2,i] = portfolio_var
    results[3,i] = results[0,i] / results[1,i]  # Store Sharpe Ratio (return / volatility)
    total_value_end = total_investment + portfolio_return
    results[4,i] = total_value_end  # Store Total Portfolio Value at end
    # Store Capital Gain/Loss as percentage
    results[5,i] = ((total_value_end - total_investment) / total_investment) * 100
    # Store weights
    for j in range(len(weights)):
        results[j+6,i] = weights[j]

# Convert results array to Pandas DataFrame
columns = ['Ret', 'Std', 'VaR', 'Sharpe', 'TotalValue', 'CapitalGainLossPct'] + [ticker + "_weight" for ticker in tickers]
results_frame = pd.DataFrame(results.T, columns=columns)

# Format the data for better readability
pd.options.display.float_format = '{:,.2f}'.format

# Print results
print("Portfolio with Highest Sharpe Ratio:")
print(results_frame.iloc[results_frame['Sharpe'].idxmax()])

# Print top 10 portfolios with the highest returns
print("\nTop 10 Portfolios with Maximum Returns:")
print(results_frame.nlargest(10, 'Ret'))


[                       0%%                      ]

[*********************100%%**********************]  5 of 5 completed


Portfolio with Highest Sharpe Ratio:
Ret                      113.51
Std                      126.17
VaR                      -94.02
Sharpe                     0.90
TotalValue           242,113.51
CapitalGainLossPct         0.05
PTT.BK_weight              0.59
AOT.BK_weight              0.00
DELTA.BK_weight            0.02
ADVANC.BK_weight           0.01
KBANK.BK_weight            0.39
Name: 48467, dtype: float64

Top 10 Portfolios with Maximum Returns:
         Ret    Std     VaR  Sharpe  TotalValue  CapitalGainLossPct  \
54146 124.85 482.69 -669.11    0.26  242,124.85                0.05   
31596 124.79 479.32 -663.62    0.26  242,124.79                0.05   
66853 123.39 329.77 -419.03    0.37  242,123.39                0.05   
31445 121.42 268.69 -320.54    0.45  242,121.42                0.05   
60151 121.36 285.75 -348.66    0.42  242,121.36                0.05   
33571 119.82 264.80 -315.74    0.45  242,119.82                0.05   
19723 119.44 322.31 -410.71    0.37  242,119.

# Adjust with max weight distribution

In [17]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm

def generate_diversified_weights(num_assets, max_weight=1.00):
    # Ensure that no weight exceeds the max_weight threshold
    weights = np.random.dirichlet(np.ones(num_assets), size=1)
    while max(weights[0]) > max_weight:
        weights = np.random.dirichlet(np.ones(num_assets), size=1)
    return weights[0]

# Tickers
tickers = ['AAPL', 'AMZN', 'PFE', 'WMT', 'CSCO']

# Fetch historical data
data = yf.download(tickers, start='2023-01-01', end='2024-12-31')['Close']

# Calculate daily returns
returns = data.pct_change().dropna()

# Number of portfolios to simulate
num_portfolios = 200000

# Daily investment and total investment
daily_investment = 1000
total_days = returns.shape[0]
total_investment = daily_investment * total_days

# Array to store results and weights
results = np.zeros((5 + len(tickers), num_portfolios))  # Added len(tickers) for weights, and extra rows for total value and capital gain/loss

for i in range(num_portfolios):
    # Generate diversified weights
    weights = generate_diversified_weights(len(tickers))
    
    # Calculate expected portfolio return and scale by total investment
    portfolio_return = np.sum(weights * returns.mean()) * total_days * daily_investment
    
    # Calculate expected portfolio volatility and scale by total investment
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * total_days, weights))) * daily_investment
    
    # Calculate portfolio VaR: Value at Risk at 95% confidence level, and scale by total investment
    portfolio_var = norm.ppf(0.05, portfolio_return, portfolio_stddev)
    
    # Store results in the results array
    results[0,i] = portfolio_return
    results[1,i] = portfolio_stddev
    results[2,i] = portfolio_var
    results[3,i] = results[0,i] / results[1,i]  # Store Sharpe Ratio (return / volatility)
    total_value_end = total_investment + portfolio_return
    results[4,i] = total_value_end  # Store Total Portfolio Value at end
    # Store weights
    for j in range(len(weights)):
        results[j+5,i] = weights[j]

# Convert results array to Pandas DataFrame
columns = ['Ret', 'Std', 'VaR', 'Sharpe', 'TotalValue'] + [ticker + "_weight" for ticker in tickers]
results_frame = pd.DataFrame(results.T, columns=columns)

# Format the data for better readability
pd.options.display.float_format = '{:,.2f}'.format

# Print results
print("Diversified Portfolio with Highest Sharpe Ratio:")
print(results_frame.iloc[results_frame['Sharpe'].idxmax()])

# Print top 10 diversified portfolios with the highest returns
print("\nTop 10 Diversified Portfolios with Maximum Returns:")
print(results_frame.nlargest(10, 'Ret'))


[*********************100%%**********************]  5 of 5 completed


Diversified Portfolio with Highest Sharpe Ratio:
Ret               481.56
Std               190.72
VaR               167.85
Sharpe              2.52
TotalValue    274,481.56
AAPL_weight         0.43
AMZN_weight         0.35
PFE_weight          0.00
WMT_weight          0.00
CSCO_weight         0.22
Name: 80547, dtype: float64

Top 10 Diversified Portfolios with Maximum Returns:
          Ret    Std    VaR  Sharpe  TotalValue  AAPL_weight  AMZN_weight  \
148652 702.74 329.32 161.07    2.13  274,702.74         0.01         0.95   
98720  702.25 331.19 157.49    2.12  274,702.25         0.01         0.95   
122325 690.87 323.78 158.29    2.13  274,690.87         0.03         0.93   
198080 690.05 322.79 159.11    2.14  274,690.05         0.06         0.92   
123545 686.80 319.35 161.52    2.15  274,686.80         0.05         0.91   
38677  686.53 321.50 157.72    2.14  274,686.53         0.02         0.92   
22963  686.06 316.04 166.21    2.17  274,686.06         0.09         0.89   
9135